In [ ]:
import os
import torch
import cv2
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim
from torch.autograd import Variable
from torchvision import models
import random
from torch.optim import lr_scheduler
import time
import copy
import torch.nn as nn
from PIL import Image

Defining Data Loader

In [ ]:
train_dataset_file = '../archive/TUSimple/train_set/training/train.txt'
val_dataset_file = '../archive/TUSimple/train_set/training/val.txt'

resize_height, resize_width = 256, 512

class Rescale():
    def __init__(self, output_size):
        assert isinstance(output_size, (tuple))
        self.output_size = output_size

    def __call__(self, sample):
        sample = cv2.resize(sample, dsize=self.output_size, interpolation=cv2.INTER_NEAREST)
        return sample

class TusimpleData(Dataset):
    def __init__(self, dataset, n_labels=3, transform=None, target_transform=None):
        self._gt_img_list = []
        self._gt_label_binary_list = []
        self.transform = transform
        self.target_transform = target_transform
        self.n_labels = n_labels

        with open(dataset, 'r') as file:
            for _info in file:
                info_tmp = _info.strip(' ').split()

                self._gt_img_list.append(info_tmp[0])
                self._gt_label_binary_list.append(info_tmp[1])

        self._shuffle()

        # DECREASE AMOUNT OF DATA
        purger = 0.1
        if purger < 1.0:
            total_size = len(self._gt_img_list)
            subset_size = int(total_size * purger)
            self._gt_img_list = self._gt_img_list[:subset_size]
            self._gt_label_binary_list = self._gt_label_binary_list[:subset_size]

    def _shuffle(self):
        c = list(zip(self._gt_img_list, self._gt_label_binary_list))
        random.shuffle(c)
        self._gt_img_list, self._gt_label_binary_list = zip(*c)

    def __len__(self):
        return len(self._gt_img_list)

    def __getitem__(self, idx):
        img = Image.open(self._gt_img_list[idx])
        label_img = cv2.imread(self._gt_label_binary_list[idx], cv2.IMREAD_COLOR)
        if self.transform:
            img = self.transform(img)
        if self.target_transform:
            label_img = self.target_transform(label_img)
        label_binary = np.zeros([label_img.shape[0], label_img.shape[1]], dtype=np.uint8)
        mask = np.where((label_img[:, :, :] != [0, 0, 0]).all(axis=2))
        label_binary[mask] = 1
        return img, label_binary

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((resize_height, resize_width)),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((resize_height, resize_width)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

target_transforms = transforms.Compose([
    Rescale((resize_width, resize_height)),
])

train_dataset = TusimpleData(train_dataset_file, transform=data_transforms['train'], target_transform=target_transforms)
val_dataset = TusimpleData(val_dataset_file, transform=data_transforms['val'], target_transform=target_transforms)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=True)

dataloaders = {
    'train': train_loader,
    'val': val_loader
}
dataset_sizes = {
    'train': len(train_loader.dataset),
    'val': len(val_loader.dataset)
}

NameError: name 'Dataset' is not defined

Defining Model

In [ ]:
class LaneLines(nn.Module):
    def __init__(self):
        super(LaneLines, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.relu = nn.ReLU()
        self.deconv1_binary = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.deconv2_binary = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.deconv3_binary = nn.ConvTranspose2d(32, 2, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        binary = self.relu(self.deconv1_binary(x))
        binary = self.relu(self.deconv2_binary(binary))
        binary = self.deconv3_binary(binary)
        binary_pred = torch.argmax(binary, dim=1, keepdim=True)
        return {
            "binary_seg_logits": binary,
            "binary_seg_pred": binary_pred
        }


Training Model

In [ ]:
print("here")

def compute_loss(net_output, binary_label):
    k_binary = 10
    loss_fn = nn.CrossEntropyLoss()
    binary_seg_logits = net_output["binary_seg_logits"]
    binary_loss = loss_fn(binary_seg_logits, binary_label)
    binary_loss = binary_loss * k_binary
    total_loss = binary_loss
    out = net_output["binary_seg_pred"]
    return total_loss, binary_loss, out

def train_loop(model, dataloader, optimizer, scheduler, device):
    model.train()
    running_loss = 0.0
    running_loss_b = 0.0

    for inputs, binarys in dataloader:
        inputs = inputs.float().to(device)
        binarys = binarys.long().to(device)
        optimizer.zero_grad()
        with torch.set_grad_enabled(True): 
            outputs = model(inputs)
            total_loss, binary_loss, out = compute_loss(outputs, binarys)
            total_loss.backward()
            optimizer.step()

        batch_size = inputs.size(0)
        running_loss += total_loss.item() * batch_size
        running_loss_b += binary_loss.item() * batch_size

    if scheduler is not None:
        scheduler.step()

    return running_loss, running_loss_b


def test_loop(model, dataloader, device):
    model.eval()  
    running_loss = 0.0
    running_loss_b = 0.0

    with torch.no_grad():
        for inputs, binarys in dataloader:
            inputs = inputs.float().to(device)
            binarys = binarys.long().to(device)

            outputs = model(inputs)
            total_loss, binary_loss, out = compute_loss(outputs, binarys)
            batch_size = inputs.size(0)
            running_loss += total_loss.item() * batch_size
            running_loss_b += binary_loss.item() * batch_size

    return running_loss, running_loss_b

here


In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LaneLines().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
num_epochs = 10
best_model_wts = copy.deepcopy(model.state_dict())
best_loss = float("inf")

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_loss, train_loss_b = train_loop(model, train_dataloader, optimizer, scheduler, DEVICE)
    print(f"Training Loss: {train_loss} | Binary Loss: {train_loss_b}")
    val_loss, val_loss_b = test_loop(model, val_dataloader, DEVICE)
    print(f"Validation Loss: {val_loss:.4f} | Binary Loss: {val_loss_b:.4f}")

    if val_loss < best_loss:
        best_loss = val_loss
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(best_model_wts, "best_model.pth")

model.load_state_dict(best_model_wts)


Epoch 1/10
Training Loss: 2626.905177116394 | Binary Loss: 2626.905177116394
Validation Loss: 86.0537 | Binary Loss: 86.0537
Epoch 2/10
Training Loss: 917.2128157615662 | Binary Loss: 917.2128157615662
Validation Loss: 72.5161 | Binary Loss: 72.5161
Epoch 3/10
Training Loss: 831.3592381477356 | Binary Loss: 831.3592381477356
Validation Loss: 68.4039 | Binary Loss: 68.4039
Epoch 4/10
Training Loss: 796.2373893260956 | Binary Loss: 796.2373893260956
Validation Loss: 65.8670 | Binary Loss: 65.8670
Epoch 5/10
Training Loss: 771.754889011383 | Binary Loss: 771.754889011383
Validation Loss: 63.9168 | Binary Loss: 63.9168
Epoch 6/10
Training Loss: 748.3979785442352 | Binary Loss: 748.3979785442352
Validation Loss: 61.6531 | Binary Loss: 61.6531
Epoch 7/10
Training Loss: 729.6348822116852 | Binary Loss: 729.6348822116852
Validation Loss: 60.6759 | Binary Loss: 60.6759
Epoch 8/10
Training Loss: 714.1959924697876 | Binary Loss: 714.1959924697876
Validation Loss: 58.7500 | Binary Loss: 58.7500
Ep

<All keys matched successfully>

In [ ]:
import os
import sys
import torch
import numpy as np
from torchvision import transforms
from PIL import Image
import cv2

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def load_test_data(img_path, transform):
    img = Image.open(img_path)
    img = transform(img)
    return img

def test():
    if not os.path.exists('test_output'):
        os.mkdir('test_output')
    img_path = '0001.png'
    resize_height, resize_width = 256, 512
    model_path = 'best_model.pth'
    data_transform = transforms.Compose([
        transforms.Resize((resize_height, resize_width)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    model = LaneLines()
    state_dict = torch.load(model_path)
    model.load_state_dict(state_dict)
    model.eval()
    model.to(DEVICE)
    inp = load_test_data(img_path, data_transform).to(DEVICE)
    inp = torch.unsqueeze(inp, dim=0)
    with torch.no_grad():
        outputs = model(inp)
    input_img = Image.open(img_path)
    input_img = input_img.resize((resize_width, resize_height))
    input_img = np.array(input_img)
    binary_pred = outputs['binary_seg_pred']
    binary_pred_np = binary_pred.detach().cpu().numpy()
    overlay = input_img.copy()
    overlay[binary_pred_np[0, 0, :, :] > 0] = [0, 0, 255]
    cv2.imwrite(os.path.join('test_output', 'input_with_prediction_overlay.jpg'), overlay)

if __name__ == "__main__":
    test()